In [ ]:
HTML(read(open("style.css"), String))

# Simple Evaluation Function
The evaluation of chess moves is a critical component in the development of chess engines.
This notebook can return a static centipawn value for each chess position. It will not look into the future and only consider the current state of the board. 

This method has been developed by Tomasz Michniewski in the Polish chess programming discussion list (progszach).


https://www.chessprogramming.org/Simplified_Evaluation_Function

In [ ]:
using Pkg
#Pkg.add("Chess")
using Chess
using BenchmarkTools
using NBInclude

## Detecting the Endgame

The is_endgame function takes in one argument, board, which is the current state of the chess board.  
The function returns a boolean value of true if the game is in the endgame and false if it is not.

The function uses a few checks to determine whether the game has reached the endgame stage or not.
1. Both sides have no queens
1. Every side which has a queen has additionally no other pieces or one minorpiece maximum.
https://www.chessprogramming.org/Simplified_Evaluation_Function 

The is_endgame function also uses some inline functions to calculate the number of minor pieces left on the board. The num_minor_pieces function takes in one argument, side, which is the color of the pieces being counted. It calculates the number of knights and bishops on the board for that side and returns the total count.

Finally, the is_in_endgame function takes in one argument, color, which is the color of the pieces being checked. It checks whether that color is in the endgame or not, based on the two checks described earlier. If the color is in the endgame, the function returns true, otherwise it returns false.

In [ ]:
function is_endgame(board::Board)::Bool
    # Inline Functions to calculate the number of minor pieces
    num_minor_pieces(side) = squarecount(knights(board, side)) + squarecount(bishops(board, side))
    # Check if Player White or Black is in endgame
    is_in_endgame(color) = isempty(queens(board, color)) || (num_minor_pieces(color) <= 1 && isempty(rooks(board, color)))
    return is_in_endgame(WHITE) && is_in_endgame(BLACK)
end

## Determining the Value of Piece on a Square:
In the game of chess, each piece has a relative value based on their ability to move and control squares on the board. Understanding the values of the pieces is essential in evaluating the position of the game and determining which pieces to exchange and which to keep. In this research paper, we will explore how the values of the pieces are determined and how they help in evaluating a chess position.

### Piecevalue:

In the game of chess, the values of the pieces are determined based on its importance and strength on the board. For example, the pawn is the weakest piece on the board, and its value is set at 100 points. In contrast, the queen is the most powerful piece, and its value is set at 900 points. The values of the other pieces fall between these two extremes, with the knight and bishop having a value of 320 and 330 points, respectively, and the rook having a value of 500 points.

In the following code, the global `Piece_Values` is set using a dictionary data structure. The dictionary maps the PieceType to its corresponding value in points.

In [ ]:
if !@isdefined(PIECE_VALUES)
    const PIECE_VALUES::Dict{PieceType, Int} = Dict([(PAWN, 100),(KNIGHT, 320),(BISHOP, 330),(ROOK, 500),(QUEEN, 900),(KING, 20000)])
end

The `get_value` function takes in 1 argument, `piece` which is a chess piece. It returns the value of the piece from global variable **PIECE_VALUES**.

In [ ]:
function get_value(piece::Piece)::Int64
    return get(PIECE_VALUES, ptype(piece), 0)
end

### Square Values:

The position of the pieces on the board can impact their value, making it essential to consider the square values when evaluating the position of a piece. To achieve this, two tables called `White_Pawn_Square_Table` and `Black_Pawn_Square_Table` are used. These tables contain the values for each type of piece and square on the board, based on the position.

 https://www.chessprogramming.org/Simplified_Evaluation_Function

In [ ]:
if !@isdefined(WHITE_PIECE_SQUARE_TABLES)
    const WHITE_PIECE_SQUARE_TABLES::Dict{PieceType, Vector{Vector{Int8}}} = Dict(
        PAWN => [
        [  0,  0,  0,  0,  0,  0,  0,  0],
        [ 50, 50, 50, 50, 50, 50, 50, 50],
        [ 10, 10, 20, 30, 30, 20, 10, 10],
        [  5,  5, 10, 25, 25, 10,  5,  5],
        [  0,  0,  0, 20, 20,  0,  0,  0],
        [  5, -5,-10,  0,  0,-10, -5,  5],
        [  5, 10, 10,-20,-20, 10, 10,  5],
        [  0,  0,  0,  0,  0,  0,  0,  0]
    ],
        KNIGHT => [
        [-50,-40,-30,-30,-30,-30,-40,-50],
        [-40,-20,  0,  0,  0,  0,-20,-40],
        [-30,  0, 10, 15, 15, 10,  0,-30],
        [-30,  5, 15, 20, 20, 15,  5,-30],
        [-30,  0, 15, 20, 20, 15,  0,-30],
        [-30,  5, 10, 15, 15, 10,  5,-30],
        [-40,-20,  0,  5,  5,  0,-20,-40],
        [-50,-40,-30,-30,-30,-30,-40,-50]
    ],
        BISHOP => [
        [-20,-10,-10,-10,-10,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5, 10, 10,  5,  0,-10],
        [-10,  5,  5, 10, 10,  5,  5,-10],
        [-10,  0, 10, 10, 10, 10,  0,-10],
        [-10, 10, 10, 10, 10, 10, 10,-10],
        [-10,  5,  0,  0,  0,  0,  5,-10],
        [-20,-10,-10,-10,-10,-10,-10,-20]
    ],
        ROOK => [
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 5, 10, 10, 10, 10, 10, 10,  5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [ 0,  0,  0,  5,  5,  0,  0,  0]
    ],
        QUEEN => [
        [-20,-10,-10, -5, -5,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5,  5,  5,  5,  0,-10],
        [ -5,  0,  5,  5,  5,  5,  0, -5],
        [  0,  0,  5,  5,  5,  5,  0, -5],
        [-10,  5,  5,  5,  5,  5,  0,-10],
        [-10,  0,  5,  0,  0,  0,  0,-10],
        [-20,-10,-10, -5, -5,-10,-10,-20]
    ],
        KING => [
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-20,-30,-30,-40,-40,-30,-30,-20],
        [-10,-20,-20,-20,-20,-20,-20,-10],
        [ 20, 20,  0,  0,  0,  0, 20, 20],
        [ 20, 30, 10,  0,  0, 10, 30, 20]
    ]
    )
    const white_king_square_end_game_table::Array{Array{Int8, 1}, 1} = [
        [-50,-40,-30,-20,-20,-30,-40,-50],
        [-30,-20,-10,  0,  0,-10,-20,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-30,  0,  0,  0,  0,-30,-30],
        [-50,-30,-30,-30,-30,-30,-30,-50]
    ]
end

In [ ]:
if !@isdefined(BLACK_PIECE_SQUARE_TABLES)
    const BLACK_PIECE_SQUARE_TABLES::Dict{PieceType, Vector{Vector{Int8}}} = Dict(
        PAWN => [
        [  0,  0,  0,  0,  0,  0,  0,  0],
        [  5, 10, 10,-20,-20, 10, 10,  5],
        [  5, -5,-10,  0,  0,-10, -5,  5],
        [  0,  0,  0, 20, 20,  0,  0,  0],
        [  5,  5, 10, 25, 25, 10,  5,  5],
        [ 10, 10, 20, 30, 30, 20, 10, 10],
        [ 50, 50, 50, 50, 50, 50, 50, 50],
        [  0,  0,  0,  0,  0,  0,  0,  0]
    ],
        KNIGHT => [
        [-50,-40,-30,-30,-30,-30,-40,-50],
        [-40,-20,  0,  5,  5,  0,-20,-40],
        [-30,  5, 10, 15, 15, 10,  5,-30],
        [-30,  0, 15, 20, 20, 15,  0,-30],
        [-30,  5, 15, 20, 20, 15,  5,-30],
        [-30,  0, 10, 15, 15, 10,  0,-30],
        [-40,-20,  0,  0,  0,  0,-20,-40],
        [-50,-40,-30,-30,-30,-30,-40,-50]
    ],
        BISHOP => [
        [-20,-10,-10,-10,-10,-10,-10,-20],
        [-10,  5,  0,  0,  0,  0,  5,-10],
        [-10, 10, 10, 10, 10, 10, 10,-10],
        [-10,  0, 10, 10, 10, 10,  0,-10],
        [-10,  5,  5, 10, 10,  5,  5,-10],
        [-10,  0,  5, 10, 10,  5,  0,-10],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-20,-10,-10,-10,-10,-10,-10,-20]
    ],
        ROOK => [
        [  0,  0,  0,  5,  5,  0,  0,  0],
        [ -5,  0,  0,  0,  0,  0,  0, -5],
        [ -5,  0,  0,  0,  0,  0,  0, -5],
        [ -5,  0,  0,  0,  0,  0,  0, -5],
        [ -5,  0,  0,  0,  0,  0,  0, -5],
        [ -5,  0,  0,  0,  0,  0,  0, -5],
        [  5, 10, 10, 10, 10, 10, 10,  5],
        [  0,  0,  0,  0,  0,  0,  0,  0]
    ],
        QUEEN => [
        [-20,-10,-10, -5, -5,-10,-10,-20],
        [-10,  0,  5,  0,  0,  0,  0,-10],
        [-10,  5,  5,  5,  5,  5,  0,-10],
        [  0,  0,  5,  5,  5,  5,  0, -5],
        [ -5,  0,  5,  5,  5,  5,  0, -5],
        [-10,  0,  5,  5,  5,  5,  0,-10],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-20,-10,-10, -5, -5,-10,-10,-20]
    ],
        KING => [
        [ 20, 30, 10,  0,  0, 10, 30, 20],
        [ 20, 20,  0,  0,  0,  0, 20, 20],
        [-10,-20,-20,-20,-20,-20,-20,-10],
        [-20,-30,-30,-40,-40,-30,-30,-20],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30]
    ]
    )
    const black_king_square_end_game_table::Array{Array{Int8, 1}, 1} = [
        [-50,-30,-30,-30,-30,-30,-30,-50],
        [-30,-30,  0,  0,  0,  0,-30,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-20,-10,  0,  0,-10,-20,-30],
        [-50,-40,-30,-20,-20,-30,-40,-50]
    ]
end

The function `get_square_value` is used to evaluate the position of a chess piece on a board. As input parameters it takes a board (board::Board), a chess piece (piece::Piece) and a playing field (square::Square) on which the piece is located. The function calculates a numeric centipawn value that describes the strength or weakness of the current position of the piece. 

The King uses two tables. The `table_opt` can be set to "end" to enforce the usage of the endgame table. This feature overrides the actual position of the board.

In [ ]:
function get_square_value(board::Board, piece::Piece, square::Square, table_opt::String="")::Int
    piece_type::PieceType = ptype(piece)
    if table_opt == "" && is_endgame(board)
        table_opt = "end"
    end

    if Chess.pcolor(piece) == WHITE
        square_table = WHITE_PIECE_SQUARE_TABLES[piece_type]
        if piece_type == KING && table_opt ==  "end"
            square_table = white_king_square_end_game_table
        end
    else
        square_table = BLACK_PIECE_SQUARE_TABLES[piece_type]
        if piece_type == KING && table_opt ==  "end"
            square_table = black_king_square_end_game_table
        end
    end

    squareString::String = tostring(square)
    x::Int8 = Int(squareString[1]) - Int('a')+1
    y::Int8 = parse(Int, squareString[2]) 
    return square_table[9 - y][x]
end


### Using the Piece Values and Square Values to Evaluate the Position of Pieces:
The `calc_score` function takes in 3 arguments, 
1. `board` which is a chess board 
1. `piece` which is a chess piece
1. `square` which is a chess square

The calc_score function is calculated by summing the value of the piece (get_value(piece)) and the square value (get_square_value(board, piece, square)). The calc_score function is used to evaluate the position of the chess pieces on the board.

In [ ]:
function calc_score(board::Board, piece::Piece, square::Square)::Int64
    return get_value(piece) + get_square_value(board, piece, square)
end

## Evaluation of a Position

### Non incremental Evaluation of a Position
The function `evaluate_position` takes in one argument, `board` which is a chess board. It returns the value of the board. This function not incrementally evaluate one move.

If the board argument is a terminal state, the terminal_evaluation function is called, which returns the final score of the game.

The evaluate_position function iterates through each square on the board and calculates the score of each piece on that square. It uses the calc_score function to determine the value of the piece on the square. The score of the piece is then multiplied by 1 if it is a white piece or -1 if it is a black piece. Finally, the scores of all the pieces on the board are summed up to give the overall score of the board.

In [ ]:
function evaluate_position(board::Board)::Int64
    if isterminal(board)
        return terminal_evaluation(board)
    end
    score = 0
    for x in 1:8,y in 1:8
        square = Square(SquareFile(x),SquareRank(y))
        piece = pieceon(board, square)
        if piece != EMPTY
            score += (Chess.pcolor(piece) == WHITE ? 1 : -1) * calc_score(board, piece, square)
        end
    end
    return score 
end

### Incrementle Evaluation of a Move
 The idea behind incremental evaluation is to evaluate the effect of a move on the board's value by only considering the changes made by the move rather than evaluating the entire board from scratch.
 
 It reduces the computational cost of evaluating a move. Since only the changes made by the move need to be evaluated, the search tree can be pruned more effectively, resulting in faster and deeper searches.

The function `valueCapturePiece` is used to update the value of a chessboard when a piece is captured by the opponent. It takes two input parameters:
1. board: a chess board
1. square: the square where the capture occurred

If the square is empty, the function returns 0, as no piece was captured. Otherwise, it uses the calc_score function to determine the value of the captured piece, which is then subtracted from the current value of the chess board. The function returns the value of the captured piece as an integer.

In [ ]:
function valueCapturePiece(board::Board, square::Square)::Int64
    capturePiece = pieceon(board, square)
    return capturePiece == EMPTY ? 0 : calc_score(board, capturePiece, square)
end

En passant is a unique chess move where a pawn can capture an opponent's pawn that has advanced two squares and landed on a square adjacent to it, as if the opposing pawn had only moved forward one square. https://www.chess.com/de/terms/en-passant-schachregeln 

The function `calcEnPassant` is used to calutate capture with en passant. As input parameters it takes a chess board `board` and a chess square `toSquare`. The toSquare parameter represents the destination square of the pawn that just moved.

Inside the function, the epsquare function is used to retrieve the en passant square on the board. If there is no en passant square, the function returns zero. Otherwise, it checks if the toSquare parameter matches the en passant square. Otherwise, it checks if the toSquare parameter matches the en passant square.  If they match, the function returns the value of the captured pawn, otherwise it returns zero.


In [ ]:
function calcEnPassant(board::Board, toSquare::Square)::Int64
    enpassantSquare = epsquare(board)
    if(enpassantSquare != SQ_NONE && enpassantSquare == toSquare)
        lastToMove = to(lastmove(board))
        return valueCapturePiece(board, lastToMove)
    end 
    return 0
end 


In the game of chess, castling is a special move that involves the king and one of the rooks. This move allows the king to move two squares towards the rook, and the rook moves to the square over which the king passed. chess.com/lessons/playing-the-game/castling 

The function `calc_castle` is used to calculate the #rook# value of a castle move. As input parameters it takes a chess board `board` and a chess move `move`. Castling Kingside does not affect the value of the rook since the value aren't change after the move (f1/f8). Casteling Queenside adds +5 to the value of the position.The function checks if the move is a queensidecastle move. If it is not, the function returns zero. Otherwise, it returns +5 Rook Square https://www.chessprogramming.org/Simplified_Evaluation_Function.

In [ ]:
function calc_castle(board::Board, piece::Piece, toSquare::Square)::Int64
    if(ptype(piece) == KING && cancastlequeenside(board, Chess.pcolor(piece)) && file(toSquare) == FILE_C)
        return  5
    end 
    return 0
end

The `evaluate_move` function takes in a board with its current static centipawn `score` and a `move`, and it returns the static centipawn score after performing the move. The function uses an incremental implementation, which means it only considers and calculates the differences between the old and new position.

The function first does the move to obtain an undo information structure, which is used to restore the board state after the move has been evaluated. The function then determines if the move results in an endgame position and adjusts the score accordingly.

The function evaluates the piece on the starting square and determines the type of piece and whether it is black or white. The score is then inverted if the piece is black, to imitate the effect of black being white.

The algorithm then subtracts the value of the piece on the old square and adds the value of the piece on the new square. The score of captured pieces is also subtracted from the score.

The function also considers special events in chess, such as castling, promotion, en passant, and entry into endgame. If the piece is a pawn, the algorithm checks for en passant moves and adjusts the score accordingly. If the move results in a promotion, the piece is updated to the promoted piece. 
If the piece is a king, the algorithm checks for castling moves and adjusts the score accordingly. Castling kingside does not affect the value of the rook, while castling queenside adds +5 to the value of the position.

The algorithm then calculates the score of the piece on the new square using the calc_score function, which takes into account the position of the piece on the board.

If the move results in an endgame position, the algorithm adjusts the score based on the value of the piece on the board. 

The score is then inverted back if the piece is black.


In [ ]:
function evaluate_move(board::Board, move::Move, score::Int64)::Int64
    undoinfo::UndoInfo = domove!(board, move)
    is_after_move_endgame::Bool = is_endgame(board)
    if isterminal(board)
        score = terminal_evaluation(board)
        undomove!(board, undoinfo)
        return score
    end
    undomove!(board, undoinfo)

    toMove::Square = to(move)
    fromMove::Square = from(move)
    piece::Piece = pieceon(board, from(move))
    pieceType::PieceType = ptype(piece)
    isBlack::Bool = Chess.pcolor(piece) == BLACK
    
    # invert score if piece is black (imitate black is white)
    score = isBlack ? -score : score
    
    # Sub Points for old Position from Piece
    score -= valueCapturePiece(board, fromMove)
    score += valueCapturePiece(board, toMove)

    # Handle en passant and promotion
    if pieceType == PAWN      
        score += calcEnPassant(board, toMove)
        if ispromotion(move)      
            piece = Piece(sidetomove(board), promotion(move))
        end
    end
    
    # Handle castling
    if pieceType == KING     
        score += calc_castle(board, piece, toMove)
    end
    
    score += calc_score(board, piece, toMove)
    
    # invert back
    score = isBlack ? -score : score
    # entry into endgame
    if is_endgame(board) != is_after_move_endgame 
        undoInfo = domove!(board, move)
        for kingPos in kings(board)
            k = pieceon(board, kingPos)
            score = Chess.pcolor(k) == BLACK ? -score : score
            if is_after_move_endgame 
                score -= get_square_value(board, k, kingPos, "middle")
                score += get_square_value(board, k, kingPos, "end")
            else
                score -= get_square_value(board, k, kingPos, "end")
                score += get_square_value(board, k, kingPos, "middle")
            end
            score = Chess.pcolor(k) == BLACK ? -score : score
        end
        undomove!(board,undoInfo )
    end
    return score
end

### terminal_evaluation Function

The terminal_evaluation function in Julia takes a Board object as an argument and returns an integer score representing the evaluation of the board in a given terminal state. The function checks if the board is in checkmate, stalemate, material draw, or rule 50 draw, and returns a score accordingly.

The function first checks if the board is in a checkmate position. If the board is in checkmate, the function returns a score of -100000 or 100000, depending on which player is in checkmate.
A repetition position occurs when the same position occurs three times in a game, and the same player has the same possible moves. If the board is a draw, it returns a score of 0.

In [ ]:
function terminal_evaluation(state::Board)::Int64
    if ischeckmate(state)
        return sidetomove(state) == WHITE ? -100000 : 100000
    elseif  isstalemate(state) ||
           ismaterialdraw(state)    || isrule50draw(state)
        return 0
    end
end